In [1]:
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [2]:
# Enable eager execution
tf.compat.v1.enable_eager_execution()

In [3]:
# GLOBAL VARIABLES
path = "data1/NITROHCS_V1.0/"
Test_Ratio = 0.2
ValRatio = 0.2
images = []
classid = []
inputShape = (32, 32, 3)
batchsizeVal = 64
epochsVal = 50

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(inputShape[0],inputShape[1]),
  batch_size=batchsizeVal)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(inputShape[0],inputShape[1]),
  batch_size=batchsizeVal)

class_names = train_ds.class_names
print(class_names)

Found 15040 files belonging to 47 classes.
Using 12032 files for training.
Found 15040 files belonging to 47 classes.
Using 3008 files for validation.
['NITROHCS-001', 'NITROHCS-002', 'NITROHCS-003', 'NITROHCS-004', 'NITROHCS-005', 'NITROHCS-006', 'NITROHCS-007', 'NITROHCS-008', 'NITROHCS-009', 'NITROHCS-010', 'NITROHCS-011', 'NITROHCS-012', 'NITROHCS-013', 'NITROHCS-014', 'NITROHCS-015', 'NITROHCS-016', 'NITROHCS-017', 'NITROHCS-018', 'NITROHCS-019', 'NITROHCS-020', 'NITROHCS-021', 'NITROHCS-022', 'NITROHCS-023', 'NITROHCS-024', 'NITROHCS-025', 'NITROHCS-026', 'NITROHCS-027', 'NITROHCS-028', 'NITROHCS-029', 'NITROHCS-030', 'NITROHCS-031', 'NITROHCS-032', 'NITROHCS-033', 'NITROHCS-034', 'NITROHCS-035', 'NITROHCS-036', 'NITROHCS-037', 'NITROHCS-038', 'NITROHCS-039', 'NITROHCS-040', 'NITROHCS-041', 'NITROHCS-042', 'NITROHCS-043', 'NITROHCS-044', 'NITROHCS-045', 'NITROHCS-046', 'NITROHCS-047']


In [6]:


# tf.compat.v1.enable_eager_execution()

# After eager execution is enabled, operations are executed as they are
# defined and Tensor objects hold concrete values, which can be accessed as
# numpy.ndarray`s through the numpy() method.
# print(type(image))
def preprocessing(image):
    
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = cv2.equalizeHist(image)
    image = image/255
    return image

# train_ds = train_ds.map(preprocessing)
# val_ds = val_ds.map(preprocessing)
for train_images in train_ds:
    map(preprocessing,train_images)
    
   
for val_images in val_ds:
    map(preprocessing,val_images)
#     print(val_images)


### Training the model



In [7]:
resnet_model = Sequential()

pretrained_model = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=inputShape,
    pooling='avg',
    classes=len(class_names),
    classifier_activation="softmax"
)

for layer in pretrained_model.layers:
    layer.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(1024, activation='relu'))
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(len(class_names), activation='softmax'))


94668760/94668760 [==============================] - 26s 0us/step


In [8]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 module_wrapper (ModuleWrapp  (None, 2048)             0         
 er)                                                             
                                                                 
 module_wrapper_1 (ModuleWra  (None, 1024)             2098176   
 pper)                                                           
                                                                 
 module_wrapper_2 (ModuleWra  (None, 512)              524800    
 pper)                                                           
                                                                 
 module_wrapper_3 (ModuleWra  (None, 512)              262656    
 pper)                                                  

In [9]:
resnet_model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


c:\Users\ALIPSA SAHOO\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [12]:
# Saving the model according to the conditions
# checkpoint = ModelCheckpoint("Resnet_100.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')

# Early Stopping is a regularization method (To minimize overfitting)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')

history = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochsVal,
    callbacks=[ early]
)

Epoch 1/50
188/188 [==============================] - 57s 300ms/step - loss: 2.5970 - accuracy: 0.2745 - val_loss: 2.6557 - val_accuracy: 0.2523
Epoch 2/50
188/188 [==============================] - 58s 306ms/step - loss: 2.4741 - accuracy: 0.2881 - val_loss: 2.5031 - val_accuracy: 0.2959
Epoch 3/50
188/188 [==============================] - 58s 307ms/step - loss: 2.3233 - accuracy: 0.3265 - val_loss: 2.3552 - val_accuracy: 0.3132
Epoch 4/50
188/188 [==============================] - 59s 311ms/step - loss: 2.1593 - accuracy: 0.3691 - val_loss: 2.3134 - val_accuracy: 0.3348
Epoch 5/50
188/188 [==============================] - 58s 310ms/step - loss: 2.0871 - accuracy: 0.3851 - val_loss: 2.0948 - val_accuracy: 0.3886
Epoch 6/50
188/188 [==============================] - 73s 388ms/step - loss: 1.9592 - accuracy: 0.4197 - val_loss: 1.9708 - val_accuracy: 0.4189
Epoch 7/50
188/188 [==============================] - 63s 334ms/step - loss: 1.8540 - accuracy: 0.4461 - val_loss: 1.9903 - val_ac

In [ ]:
plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training','validation'])
plt.title('Loss')
plt.xlabel('Epochs')

plt.figure(2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training','validation'])
plt.title('Accuracy')
plt.xlabel('Epochs')

plt.show()
score = resnet_model.evaluate(val_ds,verbose=0)
print("Test score = ",score[0])
print("Test Accuracy = ",score[1])